# Assignment 3 Report
## Introduction
The experiment aims to implement multinomial Naïve Bayesian algorithms from scratch, and try to extend the NB to get an algorithm with better performance.

## Methodology
### Coding
The project is devided into three parts:
1. Reading data,
2. vectorize the abstracts,
3. train and test model.

#### 1. Reading data
For this part, I created a `DataFrame` class, this class is designed and coded to simulate a `pan das.DataFrame`. Therefore the object of this class can be considered a dataframe, with some basic functions enabled. `train_test_split` is implemented as a instance object to split the training and testing set. 
#### 2. Count  Vectorizer
I created a CountVectorizer class for this part. `CountVectorizer` is created to simulate some basic functions provided by `sklearn.feature_extraction.CountVectorizer`. Any instance of this class is a vectorizer, use `~.fit` to feed it with texts to generate a **bag of words**, and after that use `~.transform` to transform any abstracts into a **text vector**. 
#### 3. MultinomialNB
I created a MultinomialNB class for this part. `MultinomialNB` is created to simulate some basic functions provided by `sklearn.naive_bayes.MultinomialNB`. Any instance of this class is a classifier. `~.fit` function takes train set and train the classifier by calculating parameters like **priors** and **likelihoods**, then store them as some instance variables, TF-IDF can be enabled with passing the boolean parameter `idf`. `~.predict` uses the parameters generated form `~.fit` to calculate the posterior probability. Due to technical issue, metric methods are integrated in MultinomialNB class. 
##### 3.1 IDF smoothing
IDF originally is calculated by $$\forall w\in W, IDF = \log(\frac{N}{D_w})$$However in case there are no documents that have a certain word. I made it $$\forall w\in W, IDF = \log(\frac{N+1}{D_w+1})+1$$
##### 3.2 Likelihood Smoothing
likelihood originally is calculated by $$\mathbb{P}(w_i|V) = \frac{w_i}{\sum_{x=0}^{x= |V| } x_v}$$However, in case some words have likelihoods of 0 which makes the posterior probability 0, I made it $$\mathbb{P}(w_i|V) = \frac{w_i+1}{\sum_{x=0}^{x= |V| } x_v+|V| }$$

### Experiment Designs
The experiment is implemented in two conditions. In order to make the running time reasonable, 1000 features are selected. First I try the algorithm without TF-IDF, then I try the algorithm with TF-IDF. 

Cross validation is used to evaluate overall performance of the clasifier on whole dataset. With repeatly doing cross validation 16 times on randomly shuffled dataset to make sure **chance is not acting alone**.

### Expected Outcomes
Applying TF-IDF instead of simply using the frequency of each word reflects the importance of a word to a document in a bag of word. For any word, The more documents that this word is in, the lower IDF it is. The less documents this word is in, the higher IDF it is. 

## Results and Findings
Let $H_0$be the mean accuracy of two situations are same.

Let $H_A$be the mean accuracy of two situations are different.

The repeated cross_val sections shows the sample mean accuracy of the classifier without tf-idf is 0.8859531250000001, and the $\sigma$	is 0.001247556204896173, and those of the classifier with tf-idf are 0.8859531250000001	and 0.0012475562048961737. 

The sample size of both samples are 16, therefore degree of freedom is 30. 

With two samples t-test, we can see the p-value = 0.016. This rejects our null hypothesis that the mean accuracy with no TF-IDF is same as that of using TF-IDF. With the mean accuracies returned from the repeated cross_val. With 95% confidence, we can say, the mean accuracy of using TF-IDF is significantly higher than that of not using TF-IDF.

# Code
## Initializations, Class Definitions and Type Declarations

In [ ]:
# from DataFrame import DataFrame
# from CountVectorizer import CountVectorizer
# from MultinomialNB import MultinomialNB,get_accuracy, cross_val_score
import numpy as np
from math import log, exp, sqrt, pow

rs = 42

In [ ]:
#  Copyright (c) 2021 Jäger. All rights reserved.
import string

import numpy as np
from typing import List


class CountVectorizer:

    def __init__(self, stop_words: List[str], max_features: int = None):
        self.vocabularies = []
        self.stop_words = stop_words
        self.max_features = max_features
        pass

    def fit(self, raw_documents) -> None:
        raw_documents_all_in_1 = ' '.join(raw_documents)
        all_words = np.array(self.text_process(raw_documents_all_in_1), dtype=object)
        unique, counts = np.unique(all_words, return_counts=True)
        bag_of_words = np.transpose(np.array([unique, counts], dtype=object))
        self.vocabularies = bag_of_words[bag_of_words[:, 1].argsort()]  # sort
        self.vocabularies = self.vocabularies[::-1]  # descending order
        if self.max_features is not None:
            self.vocabularies = self.vocabularies[:self.max_features]

    def transform(self, raw_documents, tf_idf=False):
        vectorized_document = np.empty(shape=(len(raw_documents), len(self.vocabularies)), dtype=object)
        for i in range(len(raw_documents)):
            words = self.text_process(raw_documents[i])
            for j in range(len(self.vocabularies)):
                vectorized_document[i, j] = words.count(self.vocabularies[:, 0][j])
        if tf_idf:
            return vectorized_document, CountVectorizer.idf(vectorized_document)
        else:
            return vectorized_document

    def text_process(self, abstract: str) -> List[str]:
        return [word.lower() for word in ''.join([char for char in abstract if char not in string.punctuation]).split()
                if
                word not in self.stop_words]

    # noinspection PyPep8Naming
    @staticmethod
    def idf(vectorized_document: np.ndarray) -> np.ndarray:
        N = len(vectorized_document)
        D = [len(vectorized_document[vectorized_document[:, j] != 0]) for j in range(vectorized_document.shape[1])]
        D = np.array(D, dtype=np.float)
        IDF = np.log(N+1 / D+1)+1
        return IDF
#  Copyright (c) 2021 Jäger. All rights reserved.

from typing import List
from numpy import str, random, array, arange, genfromtxt, ndarray, asarray, empty


class DataFrame:
    col_names: ndarray
    data: ndarray

    def __init__(self, data: ndarray = None, col_names: ndarray = None):
        self.col_names: ndarray = empty(0)
        self.data: ndarray = empty((0, 0))
        if data is not None:
            self.data = data
            if col_names is not None:
                if len(col_names) != len(self.data):
                    raise Exception('Columns names length mismatch')
                else:
                    self.col_names = asarray(col_names)
                    pass
                pass
            pass
        pass

    def get_col(self, col: int) -> ndarray:
        """
        Use index to get a specific column.

        Parameters
        ----------
        col: int
            Position index of the column.

        Returns
        -------
        ndarray
            The column at this position.
        """
        return self.data[:, col]

    def get_row(self, row: int) -> ndarray:
        """
        Use index to get a specific row.

        Parameters
        ----------
        row: int
            Position index of the row.

        Returns
        -------
        ndarray
            The row at this position.
        """
        return self.data[row]

    def get_cols(self, cols: List[int]) -> ndarray:
        """
        Use a list of indices to get a new DataFrame consists of specific columns.
        Parameters
        ----------
        cols: List[int]
            Position indices of the columns.

        Returns
        -------
        ndarray
            A DataFrame with these columns.
        """
        return self.data[:, cols]

    def get_rows(self, rows: List[int]) -> ndarray:
        """
        Use a list of indices to get a new DataFrame consists of specific rows.

        Parameters
        ----------
        rows: List[int]
            Position indices of the rows.

        Returns
        -------
        ndarray
            A DataFrame with these rows.
        """
        return self.data[rows]

    def get_sub_df(self, rows: List[int], cols: List[int]) -> ndarray:
        """
        Use two lists of indices to get a new DataFrame consists of selected rows and columns.

        Parameters
        ----------
        rows: List[int]
            Position indices of the rows.
        cols: List[int]
            Position indices of the cols.

        Returns
        -------
        ndarray
            A sub-DataFrame with selected rows and columns by position indices.
        """
        return self.data[array(rows)[:, None], array(cols)]

    def read_csv(self, filename: str, sep: str = ',', col_names: List[str] = None):
        """
        Read data from a .CSV file and save data to the caller object.

        Parameters
        ----------
        filename:str
            The path of the csv to be read.
        sep:str, default = ','
            Delimiter to use.
        col_names:List[str], optional
        """
        self.col_names = genfromtxt(filename, delimiter=sep, dtype=str, encoding='utf-8')[0]
        self.data = genfromtxt(filename, delimiter=sep, dtype=str, encoding='utf-8')[1:]
        if col_names is not None:
            if len(col_names) != len(self.data):
                raise Exception('Columns names length mismatch')
            else:
                self.col_names = asarray(col_names)
                pass
            pass
        pass

    def head(self, n: int = 5) -> ndarray:
        """
        Return the first `n` rows.

        Parameters
        ----------
        n:int, default = 5
            A positive integer which specifies the number of rows to select.

        Returns
        -------
        int
            The first `n` rows of the caller object
        """
        return self.data[0:n]
        pass

    def shape(self):
        """
        Return the shape of the caller object.
        Returns
        -------
        tuple[int]
            A tuple with lengths of each axis of the caller object.
        """
        return self.data.shape

    def train_test_split(
            self,
            test_size: float = 0.2,
            random_state: int = None,
            shuffle: bool = True
    ):
        """
        Shuffle and split the dataframe.

        Parameters
        ----------
        shuffle : bool, default = True
            Specify whether or not shuffle before splitting.
        test_size : float, default = 0.2
            The size of the testing set after splitting.
        random_state : int, optional
            Specify to reproduce same result when Running many times.

        Returns
        -------
        tuple[DataFrame, DataFrame]
            The split training set and testing set.
        """
        index_list = arange(len(self.data))
        if shuffle:
            if random_state is not None:
                random.default_rng(random_state).shuffle(index_list)
            else:
                random.default_rng().shuffle(index_list)

        border = int(len(index_list) * (1 - test_size))
        train_indices = index_list[:border]
        test_indices = index_list[border:]

        return DataFrame(self.get_rows([train_indices])), DataFrame(self.get_rows([test_indices]))
#  Copyright (c) 2021 Jäger. All rights reserved.

import numpy as np
from math import log, pow, exp


class MultinomialNB:
    def __init__(self) -> None:
        self.y_labels = []
        self.grouped_Xs = []
        self.class_priors = []
        self.likelihoods_by_class = []
        pass

    def fit(self, vec_X, y, idf=None):
        self.y_labels = np.unique(y).tolist()
        self.grouped_Xs = [vec_X[y == self.y_labels[i]] for i in range(len(self.y_labels))]
        self.class_priors = [vec_x.shape[0] / vec_X.shape[0] for vec_x in self.grouped_Xs]
        if idf is None:
            self.likelihoods_by_class = [(vec_x.sum(axis=0) + 1) / (vec_x.sum() + vec_X.shape[1]) for vec_x in
                                         self.grouped_Xs]
        else:
            self.likelihoods_by_class = [(vec_x.sum(axis=0) * idf + 1) / (sum(vec_x.sum(axis=0) * idf) + vec_X.shape[1])
                                         for vec_x in self.grouped_Xs]

    def predict(self, vectorized_samples):
        predictions = []
        for sample in vectorized_samples:
            temp = []  # list of 4 posterior_pr
            for c in range(len(self.grouped_Xs)):
                sample_likelihood = 1.0
                # log_sample_likelihood = log(sample_likelihood)
                for i in range(len(sample)):  # for word in words
                    for j in range(sample[i]):
                        # log_sample_likelihood = log_sample_likelihood + log(self.likelihoods_by_class[c][i])
                        sample_likelihood = sample_likelihood * self.likelihoods_by_class[c][i]
                # temp.append(exp(sample_likelihood + log(self.class_priors[c])))
                temp.append(sample_likelihood*self.class_priors[c])
            # print(temp)
            predictions.append(self.y_labels[temp.index(np.max(temp))])
        return predictions

def get_accuracy(estimates, true_val, verbose = False):
    n = 0
    for pair in np.array([estimates, true_val]).transpose():
        if pair[0] == pair[1]:
            n += 1
    if verbose:
        print(f"Correct: {n}\tAccuracy: {n / len(estimates):.4%}")
    return n / len(estimates)


def cross_val_score(X: np.ndarray, y: np.ndarray, idf=None, cv: int = 5):
    index_list = np.arange(X.shape[0])
    indices_folds = np.asarray(np.array_split(index_list, cv))
    results = np.zeros(cv)
    for i in range(cv):
        val_fold_indices = indices_folds[i]
        # np.delete(indices_folds, i)
        train_fold_indices = indices_folds[[j for j in range(cv) if j!=i]].flatten()
        X_val = X[val_fold_indices]
        y_val = y[val_fold_indices]
        X_train = X[train_fold_indices]
        y_train = y[train_fold_indices]
        model = MultinomialNB()
        if idf is not None:
            model.fit(X_train, y_train, idf)
        else:
            model.fit(X_train, y_train)
        predictions = model.predict(X_val)
        results[i] = get_accuracy(predictions, y_val)
    return results


<ipython-input-2-9ff57ad9f583>:53: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from numpy import str, random, array, arange, genfromtxt, ndarray, asarray, empty


## Read Data
Here The data file is expected to be at the root directory

In [ ]:
df = DataFrame()
df.read_csv('trg.csv')

In [ ]:
df.head()

array([['1', 'B',
        '"the 4 202 353 bp genome of the alkaliphilic bacterium bacillus halodurans c-125 contains 4066 predicted protein coding sequences cdss 2141 527 of which have functional assignments 1182 29 of which are conserved cdss with unknown function and 743 18 3 of which have no match to any protein database among the total cdss 88 match sequences of proteins found only in bacillus subtilis and 667 are widely conserved in comparison with the proteins of various organisms including bsubtilis the b halodurans genome contains 112 transposase genes indicating that transposases have played an important evolutionary role in horizontal gene transfer and also in internal genetic rearrangement in the genome strain c-125 lacks some of the necessary genes for competence such as coms srfa and rapc supporting the fact that competence has not been demonstrated experimentally in c-125 there is no paralog of tupa encoding teichuronopeptide which contributes to alkaliphily in the c-125 

## Train Test Split

In [ ]:
X_raw = df.get_col(2)
y = df.get_col(1)
train, val = df.train_test_split()
X_train_raw = train.get_col(2)
X_val_raw = val.get_col(2)
y_train = train.get_col(1)
y_val = val.get_col(1)

<ipython-input-2-9ff57ad9f583>:136: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return self.data[rows]


## Vectorization

Vectorization uses a list of stopwords. The stopwords should be a txt file at the root directory



In [ ]:
vectorizer = CountVectorizer(stop_words=np.genfromtxt('stopwords.txt', dtype = str), max_features=1000)

In [ ]:
vectorizer.fit(X_raw)
vec_X_train, idf_train = vectorizer.transform(X_train_raw, tf_idf=True)
vec_X, idf = vectorizer.transform(X_raw, tf_idf=True)

<ipython-input-2-9ff57ad9f583>:47: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  D = np.array(D, dtype=np.float)


In [ ]:
vec_X_train

array([[0, 0, 0, ..., 0, 0, 0],
       [3, 0, 1, ..., 0, 0, 0],
       [0, 3, 1, ..., 0, 0, 0],
       ...,
       [0, 2, 2, ..., 0, 0, 0],
       [0, 1, 6, ..., 0, 0, 0],
       [12, 3, 2, ..., 0, 0, 0]], dtype=object)

## MultinomialNB 

### Max Features = 1000, no TF_IDF

In [ ]:
model = MultinomialNB()
model.fit(vec_X_train, y_train)

In [ ]:
predictions_on_train = model.predict(vec_X_train)

In [ ]:
get_accuracy(predictions_on_train, y_train, verbose = True)

Correct: 2833	Accuracy: 88.5312%


0.8853125

In [ ]:
vec_X_val = vectorizer.transform(X_val_raw)

In [ ]:
predictions_on_val = model.predict(vec_X_val)

In [ ]:
get_accuracy(predictions_on_val, y_val, verbose = True)

Correct: 692	Accuracy: 86.5000%


0.865

#### Repeated Cross Validation
16 times repeated cross validation, dataset is shuffled randomly each time before cross validation. See `MultinomialNB` class for cross validation method implementation.

In [ ]:
rep = 16
results = np.zeros(rep)
for i in range(rep):
    index_list = np.arange(len(df.data))
    np.random.default_rng(rs*(i+1)).shuffle(index_list)
    vec_X_shuffled = vec_X[index_list]
    vec_Y_shuffled = y[index_list]
    results[i]=np.mean(cross_val_score(vec_X_shuffled, vec_Y_shuffled))
results

array([0.87425, 0.873  , 0.87475, 0.87025, 0.872  , 0.87325, 0.872  ,
       0.8685 , 0.87175, 0.8705 , 0.87275, 0.8735 , 0.87   , 0.87175,
       0.867  , 0.87225])

In [ ]:
print(f'mean: {np.mean(results)}\tstd: {np.std(results)}')

mean: 0.87171875	std: 0.0019919270663103566


### Use TF_IDF, Max Features = 1000

In [ ]:
model_tf_idf = MultinomialNB()
model_tf_idf.fit(vec_X_train, y_train, idf = idf)

In [ ]:
predictions_tf_idf = model_tf_idf.predict(vec_X_train)

In [ ]:
get_accuracy(predictions_tf_idf, y_train)

0.89625

#### Repeated Cross Validation

In [ ]:
rep = 16
results_tf_idf = np.zeros(rep)
for i in range(rep):
    index_list = np.arange(len(df.data))
    np.random.default_rng().shuffle(index_list)
    vec_X_shuffled = vec_X[index_list]
    vec_Y_shuffled = y[index_list]
    results_tf_idf[i]=np.mean(cross_val_score(vec_X_shuffled, vec_Y_shuffled, idf))
results_tf_idf

array([0.8875 , 0.8845 , 0.8845 , 0.88575, 0.884  , 0.887  , 0.88725,
       0.88675, 0.885  , 0.88675, 0.8845 , 0.885  , 0.88725, 0.88625,
       0.888  , 0.88525])

In [ ]:
print(f'mean: {np.mean(results_tf_idf)}\tstd: {np.std(results_tf_idf)}')

mean: 0.8859531250000001	std: 0.0012475562048961737


## Use scipy to do hypotheses testing

In [ ]:
from scipy.stats import sem, t
mean1 = 0.87171875	
mean2 = 0.8732656249999999	
std1=0.0019919270663103566
std2=0.0008075268474639141

n1, n2 = 16, 16
se1, se2 = std1/sqrt(n1), std2/sqrt(n2)

se1, se2 = sem(results), sem(results_tf_idf)

sed = sqrt(se1**2.0 + se2**2.0)

t_stat = (mean1 - mean2) / sed
df = 30

alpha = 0.05
cv = t.ppf(1.0 - alpha, df)

p = (1 - t.cdf(abs(t_stat), df)) * 2

P value

In [ ]:
p

0.016157454136917826